Let's run a little test to see how much we scale with nside. Maybe also do it with no DDFs to see how that impacts things

In [7]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import healpy as hp
from baseline_like import build_sched
import time
from rubin_sim.scheduler import sim_runner
from rubin_sim.scheduler.schedulers import  Core_scheduler, simple_filter_sched
from rubin_sim.scheduler.modelObservatory import Model_observatory
import pickle
import subprocess

In [2]:
nsides = [32, 64, 128]
schedulers = [build_sched(nside) for nside in nsides]

Optimizing ELAISS1
Optimizing XMM_LSS
Optimizing ECDFS
Optimizing COSMOS
Optimizing EDFS_a


Optimizing ELAISS1
Optimizing XMM_LSS
Optimizing ECDFS
Optimizing COSMOS
Optimizing EDFS_a


Optimizing ELAISS1
Optimizing XMM_LSS
Optimizing ECDFS
Optimizing COSMOS
Optimizing EDFS_a


In [3]:
run_times = []
n_obs = []

illum_limit = 40
survey_length = 30  # Days

for nside,sched in zip(nsides, schedulers):
    scheduler = Core_scheduler(sched, nside=nside)
    
    filter_sched = simple_filter_sched(illum_limit=illum_limit)
    observatory = Model_observatory(nside=nside)
    t0 = time.time()
    observatory, scheduler, observations = sim_runner(observatory, scheduler,
                                                      survey_length=survey_length,
                                                      filename=None,
                                                      filter_scheduler=filter_sched)
    t1 = time.time()
    run_times.append((t1-t0)/60.)
    n_obs.append(observations.size)
    with open('sched%i.pkl' % nside, 'wb') as f:
        pickle.dump(scheduler, f)
    

progress = 9.95%

/Users/yoachim/git_repos/rubin_sim/rubin_sim/skybrightness_pre/SkyModelPre.py:272: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn(


progress = 56.64%

/Users/yoachim/git_repos/rubin_sim/rubin_sim/utils/healpyUtils.py:331: RuntimeWarning: All-NaN slice encountered
  in_map[current_neighbors] == np.nanmax(in_map[current_neighbors])
/Users/yoachim/git_repos/rubin_sim/rubin_sim/utils/healpyUtils.py:335: UserWarning: Can not connect to any more pixels.
  warnings.warn("Can not connect to any more pixels.")


progress = 100.00%Skipped 0 observations
Flushed 2010 observations from queue for being stale
Completed 21241 observations
ran in 7 min = 0.1 hours
Writing results to  None
progress = 100.00%Skipped 0 observations
Flushed 2006 observations from queue for being stale
Completed 21237 observations
ran in 14 min = 0.2 hours
Writing results to  None
progress = 100.00%Skipped 0 observations
Flushed 1696 observations from queue for being stale
Completed 21266 observations
ran in 51 min = 0.9 hours
Writing results to  None


In [11]:
subprocess.Popen('ls -lh sched*.pkl', shell=True)

-rw-r--r--  1 yoachim  staff   1.2G Sep 16 11:17 sched128.pkl
-rw-r--r--  1 yoachim  staff   124M Sep 16 10:11 sched32.pkl
-rw-r--r--  1 yoachim  staff   347M Sep 16 10:25 sched64.pkl


In [4]:
print(run_times)

[7.7560529987017315, 14.096290449301401, 51.47799470027288]


In [5]:
print(n_obs)

[21241, 21237, 21266]
